In [1]:
from simple_lama_inpainting import SimpleLama
from PIL import Image, ImageChops, ImageDraw
import numpy as np
import cv2
import os
from torchmetrics.image.fid import FrechetInceptionDistance
import torch
from diffusers import AutoPipelineForInpainting
from diffusers.utils import load_image, make_image_grid
from PIL import Image
import numpy as np


/home/ludo/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ludo/.local/lib/python3.10/site-packages/diffusers/models/transformers/transformer_2d.py:34: FutureWarning: `Transformer2DModelOutput` is deprecated and will be removed in version 1.0.0. Importing `Transformer2DModelOutput` from `diffusers.models.transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.modeling_outputs import Transformer2DModelOutput`, instead.
  deprecate("Transformer2DModelOutput", "1.0.0", deprecation_message)
/home/ludo/.local/lib/python3.10/site-packages/diffusers/models/vq_model.py:20: FutureWarning: `VQEncoderOutput` is deprecated and will be removed in version 0.31. Importing `VQEncoderOutput` from `diffusers.models.vq_model` is deprecated and this will be

In [2]:
# 1. Lama

In [3]:
# Iterationen für Maskenmanipulation
ITERATIONS = 10

# Funktion, um die Maske zu erweitern (Dilation)
def expand_mask(mask, iterations=ITERATIONS):
    mask_array = np.array(mask)
    kernel = np.ones((3, 3), np.uint8)
    expanded_mask = mask_array
    for _ in range(iterations):
        expanded_mask = cv2.dilate(expanded_mask, kernel, iterations=1)
    return Image.fromarray(expanded_mask)

In [4]:

# Directories
images_dir = './Dataset_new/images'
masks_dir = './Dataset_new/masks'

# Initialize output
image_files = {}

# List all files in images and masks directories
image_files = [f for f in os.listdir(images_dir) if f.endswith('.jpg') and f in os.listdir(masks_dir)]

print(f'Found {len(image_files)} images')


Found 1000 images


In [5]:
# Initialisiere SimpleLama
simple_lama = SimpleLama()

In [6]:
def image_mask_generator(images_dir, masks_dir, image_files):
    """
    Generator to read images and their corresponding masks.

    Args:
    - images_dir (str): Directory containing the images.
    - masks_dir (str): Directory containing the masks.
    - image_files (list): List of filenames (same for both images and masks).

    Yields:
    - tuple: A tuple containing a PIL image and a corresponding mask (PIL image).
    """
    for fname in image_files:
        image_path = os.path.join(images_dir, fname)  # Full path to the image
        mask_path = os.path.join(masks_dir, fname)  # Full path to the mask
        
        # Load image and mask
        try:
            image = Image.open(image_path).convert('RGB')  # Convert image to RGB
            mask = Image.open(mask_path).convert('L')  # Convert mask to grayscale (L)

            mask = expand_mask(mask) # Expand mask 
            
            # Yield image and mask
            yield fname, image, mask
        except Exception as e:
            print(f"Error loading {fname}: {e}")

In [7]:
import os

def save_result(image, result, fname, results_folder):
    """
    Save the result to a directory.
    
    Args:
    - image (PIL.Image): The input image.
    - result (PIL.Image): The result of the inpainting or processing.
    - fname (str): The filename of the current image.
    - results_folder (str): Folder where results will be saved.
    
    Returns:
    - None
    """

    results_dir = f'results/{results_folder}'
    
    # Ensure the result directory exists
    os.makedirs(results_dir, exist_ok=True)
    
    
    # Create the result path for saving the image
    result_path = os.path.join(results_dir, fname)
    
    # Save the result
    result.save(result_path)
    
    # Print the success message
    print(f"Ergebnis für Bild {fname} gespeichert: {result_path}")





In [8]:
import torch
from torchvision.transforms import ToTensor, Resize, Compose
from torchmetrics.image.fid import FrechetInceptionDistance
import lpips

# Define a fixed image size
target_size = (256, 256)  # Resize all images to this size

# Preprocessing function to resize and convert images to tensors
def preprocess_images(images, target_size=(256, 256)):
    """
    Preprocess images by resizing them to the target size and converting to tensors.

    Args:
    - images (list of PIL Images): The list of images to preprocess.
    - target_size (tuple): The target size to resize images to (default is 256x256).

    Returns:
    - torch.Tensor: A tensor containing all preprocessed images, stacked in a batch.
    """
    transform = Compose([
        Resize(target_size),  # Resize the images to the target size
        ToTensor(),  # Convert the images to tensor
    ])

    # Apply the transformation to each image in the list
    image_tensors = [transform(image) for image in images]

    # Stack the tensors into a single batch
    stacked_tensor = torch.stack(image_tensors)
    
    return stacked_tensor


# Error calculation function to compute the FID score
def calculate_fid(real_images_tensor, infilled_images_tensor):
    """
    Calculate the Fréchet Inception Distance (FID) between real and Lama generated images.

    Args:
    - real_images_tensor (tensor): The tensor of real images.
    - infilled_images_tensor (list of PIL Images): The tensor of infilled images.

    Returns:
    - float: The FID score.
    """
    # Initialize the FID metric
    fid = FrechetInceptionDistance(normalize=True)

    # Update the FID with the preprocessed images
    fid.update(real_images_tensor, real=True)
    fid.update(infilled_images_tensor, real=False)

    # Calculate and return the FID score
    return float(fid.compute())


# LPIPS calculation function to compute the LPIPS score
def calculate_lpips(real_images_tensor, infilled_images_tensor):
    """
    Calculate the Learned Perceptual Image Patch Similarity (LPIPS) between real and Lama generated images.

    Args:
    - real_images_tensor (tensor): The tensor of real images.
    - infilled_images_tensor (list of PIL Images): The tensor of infilled images.

    Returns:
    - float: The LPIPS score.
    """

    # Load the LPIPS model
    loss_fn = lpips.LPIPS(net='alex')  # Using AlexNet architecture for LPIPS


    # Compute the LPIPS score
    lpips_score = loss_fn(real_images_tensor, infilled_images_tensor)

    # Return the average LPIPS score
    return lpips_score.mean().item()



In [9]:
real_images = []
lama_images = []

# Schleife durch alle Bilder und Masken

image_gen = image_mask_generator(images_dir, masks_dir, image_files)
for fname, image, mask in image_gen:

    result = simple_lama(image, mask) # Infill mit Lama und der neuen Maske

    real_images.append(image)
    lama_images.append(result)

    # Append and save result
    save_result(image, result, fname, 'lama')
    

Ergebnis für Bild ADE_train_00003173.jpg gespeichert: results/lama/ADE_train_00003173.jpg
Ergebnis für Bild ADE_train_00023741.jpg gespeichert: results/lama/ADE_train_00023741.jpg
Ergebnis für Bild ADE_train_00022732.jpg gespeichert: results/lama/ADE_train_00022732.jpg
Ergebnis für Bild ADE_train_00001504.jpg gespeichert: results/lama/ADE_train_00001504.jpg
Ergebnis für Bild ADE_frame_00000204.jpg gespeichert: results/lama/ADE_frame_00000204.jpg
Ergebnis für Bild ADE_train_00003171.jpg gespeichert: results/lama/ADE_train_00003171.jpg
Ergebnis für Bild ADE_train_00012219.jpg gespeichert: results/lama/ADE_train_00012219.jpg
Ergebnis für Bild ADE_train_00001961.jpg gespeichert: results/lama/ADE_train_00001961.jpg
Ergebnis für Bild ADE_train_00024430.jpg gespeichert: results/lama/ADE_train_00024430.jpg
Ergebnis für Bild ADE_train_00001716.jpg gespeichert: results/lama/ADE_train_00001716.jpg
Ergebnis für Bild ADE_train_00001676.jpg gespeichert: results/lama/ADE_train_00001676.jpg
Ergebnis f

In [10]:
# Preprocess the images (resize and convert to tensors)
real_images_tensor = preprocess_images(real_images, target_size)
lama_images_tensor = preprocess_images(lama_images, target_size)

fid_score = calculate_fid(real_images_tensor, lama_images_tensor)

lpips_score = calculate_lpips(real_images_tensor, lama_images_tensor)

print(f"FID: {fid_score}")
print(f"LPIPS: {lpips_score}")

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/home/ludo/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ludo/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/ludo/.conda/envs/lama/lib/python3.10/site-packages/lpips/weights/v0.1/alex.pth
FID: 31.581621170043945
LPIPS: 0.1356128454208374


In [11]:
# Baseline - Opencv

In [12]:
def image_mask_generator_opencv(images_dir, masks_dir, image_files):
    """
    Generator to read images and their corresponding masks.

    Args:
    - images_dir (str): Directory containing the images.
    - masks_dir (str): Directory containing the masks.
    - image_files (list): List of filenames (same for both images and masks).

    Yields:
    - tuple: A tuple containing a PIL image and a corresponding mask (PIL image).
    """
    for fname in image_files:
        image_path = os.path.join(images_dir, fname)  # Full path to the image
        mask_path = os.path.join(masks_dir, fname)  # Full path to the mask
        
        # Load image and mask
        try:
            image = Image.open(image_path) 
            mask = Image.open(mask_path).convert('L')  

            mask = expand_mask(mask) # Expand mask 
            
            # Yield image and mask
            yield fname, image, mask
        except Exception as e:
            print(f"Error loading {fname}: {e}")

In [13]:
def inpaint_with_opencv(image, mask, inpaint_radius=3, method=cv2.INPAINT_TELEA):
    """
    Perform inpainting using OpenCV, ensuring correct color conversions.

    Args:
    - image (PIL.Image): The input image.
    - mask (PIL.Image): The binary mask (grayscale).
    - inpaint_radius (int): Radius of the inpainting.
    - method: Inpainting method (cv2.INPAINT_TELEA or cv2.INPAINT_NS).

    Returns:
    - PIL.Image: The inpainted image with correct colors.
    """
    # Convert PIL image to OpenCV format (RGB -> BGR)
    image_array = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    mask_array = np.array(mask)

    # Ensure mask is binary (0 and 255)
    _, mask_array = cv2.threshold(mask_array, 1, 255, cv2.THRESH_BINARY)

    # Perform inpainting
    inpainted_image = cv2.inpaint(image_array, mask_array, inpaint_radius, method)

    # Convert back to PIL format (BGR -> RGB)
    inpainted_image = cv2.cvtColor(inpainted_image, cv2.COLOR_BGR2RGB)
    return Image.fromarray(inpainted_image)


In [14]:
real_images = []
opencv_images = []

# Schleife durch alle Bilder und Masken

image_gen = image_mask_generator_opencv(images_dir, masks_dir, image_files)
for fname, image, mask in image_gen:

    result = inpaint_with_opencv(image, mask) # Infill mit Lama und der neuen Maske

    real_images.append(image)
    opencv_images.append(result)

    # Append and save result
    save_result(image, result, fname, 'opencv')

Ergebnis für Bild ADE_train_00003173.jpg gespeichert: results/opencv/ADE_train_00003173.jpg
Ergebnis für Bild ADE_train_00023741.jpg gespeichert: results/opencv/ADE_train_00023741.jpg
Ergebnis für Bild ADE_train_00022732.jpg gespeichert: results/opencv/ADE_train_00022732.jpg
Ergebnis für Bild ADE_train_00001504.jpg gespeichert: results/opencv/ADE_train_00001504.jpg
Ergebnis für Bild ADE_frame_00000204.jpg gespeichert: results/opencv/ADE_frame_00000204.jpg
Ergebnis für Bild ADE_train_00003171.jpg gespeichert: results/opencv/ADE_train_00003171.jpg
Ergebnis für Bild ADE_train_00012219.jpg gespeichert: results/opencv/ADE_train_00012219.jpg
Ergebnis für Bild ADE_train_00001961.jpg gespeichert: results/opencv/ADE_train_00001961.jpg
Ergebnis für Bild ADE_train_00024430.jpg gespeichert: results/opencv/ADE_train_00024430.jpg
Ergebnis für Bild ADE_train_00001716.jpg gespeichert: results/opencv/ADE_train_00001716.jpg
Ergebnis für Bild ADE_train_00001676.jpg gespeichert: results/opencv/ADE_train_0

In [15]:
# Preprocess the images (resize and convert to tensors)
real_images_tensor = preprocess_images(real_images, target_size)
opencv_images_tensor = preprocess_images(opencv_images, target_size)

fid_score = calculate_fid(real_images_tensor, opencv_images_tensor)

lpips_score = calculate_lpips(real_images_tensor, opencv_images_tensor)

print(f"FID: {fid_score}")
print(f"LPIPS: {lpips_score}")

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/ludo/.conda/envs/lama/lib/python3.10/site-packages/lpips/weights/v0.1/alex.pth
FID: 37.67695236206055
LPIPS: 0.12878979742527008


In [16]:
## Stable Diffusion

In [17]:
import torch
from diffusers import AutoPipelineForInpainting
from diffusers.utils import load_image, make_image_grid
from PIL import Image
import numpy as np

# Pipeline initialisieren
pipeline = AutoPipelineForInpainting.from_pretrained(
    "kandinsky-community/kandinsky-2-2-decoder-inpaint", torch_dtype=torch.float16
)
pipeline.enable_model_cpu_offload()

pipeline.to("cuda")




Loading pipeline components...: 100%|██████████| 6/6 [00:00<00:00,  6.25it/s]
It seems like you have activated model offloading by calling `enable_model_cpu_offload`, but are now manually moving the pipeline to GPU. It is strongly recommended against doing so as memory gains from offloading are likely to be lost. Offloading automatically takes care of moving the individual components unet, scheduler, movq, prior_prior, prior_image_encoder, prior_text_encoder, prior_tokenizer, prior_scheduler, prior_image_processor to GPU when needed. To make sure offloading works as expected, you should consider moving the pipeline back to CPU: `pipeline.to('cpu')` or removing the move altogether if you use offloading.


KandinskyV22InpaintCombinedPipeline {
  "_class_name": "KandinskyV22InpaintCombinedPipeline",
  "_diffusers_version": "0.29.0",
  "_name_or_path": "kandinsky-community/kandinsky-2-2-decoder-inpaint",
  "movq": [
    "diffusers",
    "VQModel"
  ],
  "prior_image_encoder": [
    "transformers",
    "CLIPVisionModelWithProjection"
  ],
  "prior_image_processor": [
    "transformers",
    "CLIPImageProcessor"
  ],
  "prior_prior": [
    "diffusers",
    "PriorTransformer"
  ],
  "prior_scheduler": [
    "diffusers",
    "UnCLIPScheduler"
  ],
  "prior_text_encoder": [
    "transformers",
    "CLIPTextModelWithProjection"
  ],
  "prior_tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "scheduler": [
    "diffusers",
    "DDPMScheduler"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ]
}

In [18]:
from torchvision import transforms


def sd_inpaint(image, mask):

    # Generator mit festem Seed für reproduzierbare Ergebnisse
    generator = torch.Generator("cuda").manual_seed(92)

    # Define a transformation to convert PIL.Image to a PyTorch tensor
    transform = transforms.ToTensor()
    
    # Convert image and mask to tensors
    image_tensor = transform(image).unsqueeze(0)  # Add batch dimension
    mask_tensor = transform(mask).unsqueeze(0)
    
    # Move tensors to the GPU
    image_tensor = image_tensor.to("cuda")
    mask_tensor = mask_tensor.to("cuda")

    
    # Inpainting mit Stable Diffusion
    result = pipeline(
        prompt='background',
        image=image,
        mask_image=mask,
        generator=generator
    ).images[0]

    return result

    

In [19]:
real_images = []
sd_images = []

# Schleife durch alle Bilder und Masken

image_gen = image_mask_generator(images_dir, masks_dir, image_files)
for fname, image, mask in image_gen:

    result = sd_inpaint(image, mask) # Infill mit sd und neuer maske

    real_images.append(image)
    sd_images.append(result)

    # Append and save result
    save_result(image, result, fname, 'sd')

100%|██████████| 100/100 [00:07<00:00, 14.10it/s]


Ergebnis für Bild ADE_train_00003173.jpg gespeichert: results/sd/ADE_train_00003173.jpg


100%|██████████| 100/100 [00:06<00:00, 16.01it/s]


Ergebnis für Bild ADE_train_00023741.jpg gespeichert: results/sd/ADE_train_00023741.jpg


100%|██████████| 100/100 [00:06<00:00, 16.06it/s]


...

Ergebnis für Bild ADE_train_00012232.jpg gespeichert: results/sd/ADE_train_00012232.jpg


100%|██████████| 100/100 [00:06<00:00, 14.42it/s]


Ergebnis für Bild ADE_train_00020863.jpg gespeichert: results/sd/ADE_train_00020863.jpg


100%|██████████| 100/100 [00:06<00:00, 14.48it/s]


Ergebnis für Bild ADE_train_00009499.jpg gespeichert: results/sd/ADE_train_00009499.jpg


In [20]:
# Preprocess the images (resize and convert to tensors)
real_images_tensor = preprocess_images(real_images, target_size)
sd_images_tensor = preprocess_images(sd_images, target_size)

fid_score = calculate_fid(real_images_tensor, sd_images_tensor)

lpips_score = calculate_lpips(real_images_tensor, sd_images_tensor)

print(f"FID: {fid_score}")
print(f"LPIPS: {lpips_score}")

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/ludo/.conda/envs/lama/lib/python3.10/site-packages/lpips/weights/v0.1/alex.pth
FID: 49.47455978393555
LPIPS: 0.14442600309848785
